In [1]:
!nvidia-smi

Sun Jan 25 12:22:47 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# Задание 1

In [3]:
%%writefile assignment4_task1.cu

#include <iostream>        // Стандартный ввод-вывод
#include <vector>          // Поддержка контейнера динамических массивов (векторов)
#include <chrono>          // Библиотека для точного измерения времени на CPU
#include <random>          // Генератор случайных чисел
#include <cuda_runtime.h>  // Основной API CUDA для работы с видеокартой

#define N 100000           // Размер массива (100 тысяч элементов)
#define BLOCK_SIZE 256     // Количество потоков в одном блоке CUDA
#define RAND_MIN_VAL -10000 // Минимальное значение случайного числа
#define RAND_MAX_VAL 10000  // Максимальное значение случайного числа

// CUDA-ядро: функция, которая выполняется параллельно на множестве ядер GPU
__global__ void sumKernel(int* d_array, unsigned long long* d_result, int n) {
    // Вычисляем глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, не выходит ли индекс за пределы массива
    if (idx < n) {
        // Атомарное сложение: безопасно прибавляет значение элемента к общей сумме.
        // atomicAdd гарантирует, что разные потоки не перезапишут данные друг друга одновременно.
        atomicAdd(d_result, (unsigned long long)d_array[idx]);
    }
}

int main() {
    // ---------------- ИНИЦИАЛИЗАЦИЯ СЛУЧАЙНОГО МАССИВА ----------------
    std::vector<int> h_array(N); // Создаем массив на хосте (CPU) размера N

    std::random_device rd;  // Источник энтропии для генератора
    std::mt19937 gen(rd()); // Генератор "Вихрь Мерсенна"
    std::uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL); // Равномерное распределение чисел

    // Заполняем массив случайными числами
    for (int i = 0; i < N; i++)
        h_array[i] = dist(gen);

    // ----------- ВЫВОД ЧАСТИ МАССИВА ДЛЯ ПРОВЕРКИ -----------
    std::cout << "First 10: ";
    for (int i = 0; i < 10; i++) std::cout << h_array[i] << " ";
    std::cout << "\nLast 10: ";
    for (int i = N - 10; i < N; i++) std::cout << h_array[i] << " ";
    std::cout << "\n";

    // ---------------- СУММИРОВАНИЕ НА CPU ----------------
    long long cpu_sum = 0;
    auto cpu_start = std::chrono::high_resolution_clock::now(); // Засекаем время начала

    for (int i = 0; i < N; i++)
        cpu_sum += h_array[i]; // Обычный цикл суммирования

    auto cpu_end = std::chrono::high_resolution_clock::now();   // Засекаем время конца

    // Вычисляем длительность в миллисекундах
    std::chrono::duration<double, std::milli> cpu_time = cpu_end - cpu_start;

    // ---------------- СУММИРОВАНИЕ НА GPU ----------------
    int* d_array;                   // Указатель для массива на видеокарте
    unsigned long long* d_result;   // Указатель для результата на видеокарте
    unsigned long long gpu_sum = 0; // Переменная для хранения итоговой суммы на CPU

    // Выделяем память в Глобальной памяти видеокарты (VRAM)
    cudaMalloc(&d_array, N * sizeof(int));
    cudaMalloc(&d_result, sizeof(unsigned long long));

    // Копируем данные из оперативной памяти (Host) в видеопамять (Device)
    cudaMemcpy(d_array, h_array.data(), N * sizeof(int), cudaMemcpyHostToDevice);
    // Обнуляем переменную результата на видеокарте
    cudaMemset(d_result, 0, sizeof(unsigned long long));

    // Рассчитываем количество блоков: (N / размер_блока) с округлением вверх
    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Создаем события CUDA для измерения времени выполнения ядра
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start); // Запись события "старт"
    // Запуск ядра: <<<количество_блоков, потоков_в_блоке>>>
    sumKernel<<<blocks, BLOCK_SIZE>>>(d_array, d_result, N);
    cudaEventRecord(stop);  // Запись события "стоп"

    // Копируем результат вычислений обратно с видеокарты на CPU
    cudaMemcpy(&gpu_sum, d_result, sizeof(unsigned long long), cudaMemcpyDeviceToHost);

    // Ждем завершения всех операций на GPU перед расчетом времени
    cudaEventSynchronize(stop);

    float gpu_time = 0.0f;
    // Считаем разницу времени между событиями start и stop
    cudaEventElapsedTime(&gpu_time, start, stop);

    // ---------------- ВЫВОД РЕЗУЛЬТАТОВ ----------------
    std::cout << "\nArray size: " << N << std::endl;
    std::cout << "CPU sum: " << cpu_sum << std::endl;
    std::cout << "GPU sum: " << (long long)gpu_sum << std::endl;
    std::cout << "CPU time (ms): " << cpu_time.count() << std::endl;
    std::cout << "GPU time (ms): " << gpu_time << std::endl;

    // Освобождаем выделенную видеопамять
    cudaFree(d_array);
    cudaFree(d_result);

    return 0; // Завершение программы
}

Writing assignment4_task1.cu


In [4]:
!nvcc -arch=sm_75 assignment4_task1.cu -o task1
!./task1

First 10: 6717 8088 -9271 -1762 8264 -8391 329 890 1958 5912 
Last 10: -4735 -7908 -4457 -5747 8055 -3145 2887 -1930 1777 9958 

Array size: 100000
CPU sum: -1989944
GPU sum: -1989944
CPU time (ms): 0.268119
GPU time (ms): 0.255904


# Задание 2

In [12]:
%%writefile assignment4_task2.cu

#include <iostream>
#include <vector>
#include <chrono>
#include <random>
#include <cuda_runtime.h>

#define N 1000000           // Общее количество элементов в массиве
#define BLOCK_SIZE 256      // Количество потоков в одном CUDA-блоке
#define RAND_MIN_VAL 1      // Минимальное значение для генерации чисел
#define RAND_MAX_VAL 100    // Максимальное значение для генерации чисел

// ---------------- Kernel 1: Сканирование на уровне блоков ----------------
// Выполняет префиксную сумму внутри каждого блока и сохраняет общую сумму блока
__global__ void blockScanKernel(int* d_input, int* d_output, int* d_block_sums, int n) {
    // Выделяем разделяемую память (shared memory) для быстрой работы внутри блока
    __shared__ int temp[BLOCK_SIZE];

    int tid = threadIdx.x;                           // Индекс потока внутри блока
    int gid = blockIdx.x * blockDim.x + tid;         // Глобальный индекс потока

    // Копируем данные из глобальной памяти в быструю разделяемую память
    // Если глобальный индекс за пределами массива, заполняем нулем
    temp[tid] = (gid < n) ? d_input[gid] : 0;
    __syncthreads();                                 // Ждем, пока все потоки загрузят данные

    // Алгоритм инклюзивного сканирования Hillis–Steele
    // На каждой итерации шаг (offset) удваивается
    for (int offset = 1; offset < blockDim.x; offset <<= 1) {
        int val = 0;
        if (tid >= offset)                           // Если у потока есть сосед слева на расстоянии offset
            val = temp[tid - offset];                // Берем его значение
        __syncthreads();                             // Синхронизация перед записью, чтобы не затереть нужные данные
        temp[tid] += val;                            // Суммируем
        __syncthreads();                             // Синхронизация перед следующей итерацией
    }

    // Записываем результат внутриблочного сканирования в выходной массив
    if (gid < n)
        d_output[gid] = temp[tid];

    // Последний поток каждого блока сохраняет полную сумму блока в отдельный массив
    // Это нужно для того, чтобы потом скорректировать значения в следующих блоках
    if (tid == blockDim.x - 1)
        d_block_sums[blockIdx.x] = temp[tid];
}

// ---------------- Kernel 2: Добавление сумм блоков к элементам ----------------
// Берет результаты сканирования сумм блоков и прибавляет их к соответствующим блокам
__global__ void addBlockSumsKernel(int* d_output, int* d_block_sums_scan, int n) {
    int gid = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс потока

    // Первый блок (index 0) не нуждается в коррекции, также проверяем границы массива
    if (blockIdx.x == 0 || gid >= n) return;

    // Берем префиксную сумму всех предыдущих блоков
    int add_val = d_block_sums_scan[blockIdx.x - 1];
    // Прибавляем эту сумму к текущему элементу
    d_output[gid] += add_val;
}

int main() {
    // ---------------- ИНИЦИАЛИЗАЦИЯ МАССИВОВ ----------------
    // Хост-векторы для входных данных, результата CPU и результата GPU
    std::vector<int> h_input(N), h_output_cpu(N), h_output_gpu(N);

    // Настройка генератора случайных чисел
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    // Заполнение входного массива случайными числами
    for (int i = 0; i < N; i++)
        h_input[i] = dist(gen);

    // ---------------- ПРЕФИКСНАЯ СУММА НА CPU ----------------
    auto cpu_start = std::chrono::high_resolution_clock::now(); // Замер времени начала
    h_output_cpu[0] = h_input[0];
    for (int i = 1; i < N; i++)
        h_output_cpu[i] = h_output_cpu[i - 1] + h_input[i]; // Последовательный алгоритм
    auto cpu_end = std::chrono::high_resolution_clock::now();   // Замер времени окончания
    std::chrono::duration<double, std::milli> cpu_time = cpu_end - cpu_start;

    // ---------------- ПРЕФИКСНАЯ СУММА НА GPU ----------------
    int *d_input, *d_output, *d_block_sums, *d_block_sums_scan;
    int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE; // Вычисляем необходимое кол-во блоков

    // Выделение видеопамяти под входные, выходные данные и промежуточные суммы блоков
    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_output, N * sizeof(int));
    cudaMalloc(&d_block_sums, numBlocks * sizeof(int));
    cudaMalloc(&d_block_sums_scan, numBlocks * sizeof(int));

    // Копирование входных данных с хоста (RAM) на девайс (VRAM)
    cudaMemcpy(d_input, h_input.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    // Создание событий CUDA для точного замера времени выполнения ядер
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start); // Фиксация времени старта

    // ЭТАП 1: Запуск ядра для локального сканирования внутри каждого блока

    blockScanKernel<<<numBlocks, BLOCK_SIZE>>>(d_input, d_output, d_block_sums, N);

    // ЭТАП 2: Сканирование сумм блоков
    // В данном коде это делается на CPU, так как количество блоков значительно меньше N
    std::vector<int> h_block_sums(numBlocks);
    // Копируем частичные суммы блоков обратно на CPU
    cudaMemcpy(h_block_sums.data(), d_block_sums, numBlocks * sizeof(int), cudaMemcpyDeviceToHost);

    std::vector<int> h_block_sums_scan(numBlocks);
    h_block_sums_scan[0] = h_block_sums[0];
    for (int i = 1; i < numBlocks; i++)
        h_block_sums_scan[i] = h_block_sums_scan[i - 1] + h_block_sums[i]; // Префиксная сумма самих блоков

    // Копируем отсканированные суммы блоков обратно на GPU
    cudaMemcpy(d_block_sums_scan, h_block_sums_scan.data(), numBlocks * sizeof(int), cudaMemcpyHostToDevice);

    // ЭТАП 3: Финальная коррекция
    // Добавляем накопленную сумму предыдущих блоков к элементам текущего блока
    addBlockSumsKernel<<<numBlocks, BLOCK_SIZE>>>(d_output, d_block_sums_scan, N);

    // Копируем финальный результат с GPU на CPU
    cudaMemcpy(h_output_gpu.data(), d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    cudaEventRecord(stop);      // Фиксация времени окончания
    cudaEventSynchronize(stop); // Ожидание завершения всех операций на GPU

    float gpu_time = 0.0f;
    cudaEventElapsedTime(&gpu_time, start, stop); // Расчет затраченного времени в мс

    // ---------------- ВЫВОД РЕЗУЛЬТАТОВ ----------------
    int print_count = 10;
    std::cout << "\nInput array - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_input[i] << " ";
    std::cout << "\nInput array - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_input[i] << " ";

    std::cout << "\n\nCPU prefix sum - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_output_cpu[i] << " ";
    std::cout << "\nCPU prefix sum - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_output_cpu[i] << " ";

    std::cout << "\n\nGPU prefix sum - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_output_gpu[i] << " ";
    std::cout << "\nGPU prefix sum - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_output_gpu[i] << " ";

    std::cout << "\n\nArray size: " << N << std::endl;
    std::cout << "CPU time (ms): " << cpu_time.count() << std::endl;
    std::cout << "GPU time (ms): " << gpu_time << std::endl;

    // Освобождение выделенной видеопамяти
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_block_sums);
    cudaFree(d_block_sums_scan);

    return 0;
}

Overwriting assignment4_task2.cu


In [13]:
!nvcc -arch=sm_75 assignment4_task2.cu -o task2
!./task2


Input array - first 10: 11 91 78 20 59 29 77 100 24 3 
Input array - last 10: 11 30 2 23 46 46 81 90 19 48 

CPU prefix sum - first 10: 11 102 180 200 259 288 365 465 489 492 
CPU prefix sum - last 10: 50497629 50497659 50497661 50497684 50497730 50497776 50497857 50497947 50497966 50498014 

GPU prefix sum - first 10: 11 102 180 200 259 288 365 465 489 492 
GPU prefix sum - last 10: 50497629 50497659 50497661 50497684 50497730 50497776 50497857 50497947 50497966 50498014 

Array size: 1000000
CPU time (ms): 7.21465
GPU time (ms): 1.31226


# Задание 3

In [14]:
%%writefile assignment4_task3.cu

#include <iostream>
#include <vector>
#include <chrono>
#include <random>
#include <cuda_runtime.h>

#define N 1000000           // Общее количество элементов в массиве (1 миллион)
#define BLOCK_SIZE 256      // Количество потоков в одном CUDA-блоке
#define RAND_MIN_VAL -1000  // Нижняя граница для генерации случайных чисел
#define RAND_MAX_VAL 1000   // Верхняя граница для генерации случайных чисел

// ---------------- GPU Kernel ----------------
// Ядро CUDA: выполняется параллельно на видеокарте
__global__ void processKernel(int* d_array, int n) {
    // Вычисляем уникальный глобальный индекс потока
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, не выходит ли индекс за пределы массива
    if (gid < n) {
        // Выполняем полезную работу: умножаем значение элемента на 2
        d_array[gid] *= 2;
    }
}

int main() {
    // ---------------- ИНИЦИАЛИЗАЦИЯ МАССИВОВ ----------------
    // Резервируем память на хосте (CPU) для исходного массива и результатов разных типов обработки
    std::vector<int> h_array(N), h_cpu(N), h_gpu(N), h_hybrid(N);

    // Подготовка генератора случайных чисел
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    // Заполнение исходного массива случайными данными
    for (int i = 0; i < N; i++) {
        h_array[i] = dist(gen);
    }

    // ---------------- ОБРАБОТКА НА CPU ----------------
    h_cpu = h_array; // Копируем данные в массив для обработки на процессоре
    auto cpu_start = std::chrono::high_resolution_clock::now(); // Засекаем время начала
    for (int i = 0; i < N; i++) {
        h_cpu[i] *= 2; // Последовательное умножение каждого элемента
    }
    auto cpu_end = std::chrono::high_resolution_clock::now();   // Засекаем время окончания
    // Вычисляем длительность в миллисекундах
    std::chrono::duration<double, std::milli> cpu_time = cpu_end - cpu_start;

    // ---------------- ОБРАБОТКА НА GPU ----------------
    h_gpu = h_array; // Подготовка массива для GPU-теста
    int* d_array;    // Указатель для памяти на видеокарте

    // Выделяем память в VRAM и копируем туда данные с хоста
    cudaMalloc(&d_array, N * sizeof(int));
    cudaMemcpy(d_array, h_gpu.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    // Расчет количества блоков, необходимых для покрытия всего массива N
    int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Настройка событий CUDA для точного замера времени работы видеокарты
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start); // Старт записи времени

    // Запуск вычислительного ядра на GPU
    processKernel<<<numBlocks, BLOCK_SIZE>>>(d_array, N);

    cudaEventRecord(stop);      // Остановка записи времени
    cudaEventSynchronize(stop); // Ждем завершения всех операций на GPU

    // Копируем обработанные данные обратно в оперативную память (RAM)
    cudaMemcpy(h_gpu.data(), d_array, N * sizeof(int), cudaMemcpyDeviceToHost);

    float gpu_time = 0.0f;
    cudaEventElapsedTime(&gpu_time, start, stop); // Получаем время работы в мс
    cudaFree(d_array); // Освобождаем память на видеокарте

    // ---------------- ГИБРИДНАЯ ОБРАБОТКА (CPU + GPU) ----------------
    int split = N / 2;  // Определяем точку разделения: половина на CPU, половина на GPU
    h_hybrid = h_array; // Копируем исходные данные

    // Фиксируем время начала всей гибридной операции
    auto hybrid_start = std::chrono::high_resolution_clock::now();

    // Часть 1: CPU обрабатывает первую половину массива
    for (int i = 0; i < split; i++) {
        h_hybrid[i] *= 2;
    }

    // Часть 2: GPU обрабатывает вторую половину массива
    int* d_hybrid;
    // Выделяем память только под оставшуюся половину (N - split)
    cudaMalloc(&d_hybrid, (N - split) * sizeof(int));
    // Копируем вторую часть данных на девайс, начиная со смещения split
    cudaMemcpy(d_hybrid, &h_hybrid[split], (N - split) * sizeof(int), cudaMemcpyHostToDevice);

    // Рассчитываем блоки для гибридной части
    int blocks_hybrid = ((N - split) + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Запуск ядра для обработки второй половины
    processKernel<<<blocks_hybrid, BLOCK_SIZE>>>(d_hybrid, N - split);
    cudaDeviceSynchronize(); // Синхронизация устройства

    // Копируем результат GPU-части обратно в массив h_hybrid по нужному адресу
    cudaMemcpy(&h_hybrid[split], d_hybrid, (N - split) * sizeof(int), cudaMemcpyDeviceToHost);
    auto hybrid_end = std::chrono::high_resolution_clock::now(); // Конец гибридной обработки

    // Рассчитываем общее время гибридного подхода (включая копирование и работу CPU)
    float hybrid_time = std::chrono::duration<double, std::milli>(hybrid_end - hybrid_start).count();
    cudaFree(d_hybrid); // Освобождаем выделенную память

    // ---------------- ПРОВЕРКА И ВЫВОД РЕЗУЛЬТАТОВ ----------------
    int print_count = 10; // Количество элементов для предпросмотра

    // Вывод первых и последних элементов исходного массива
    std::cout << "Original array - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_array[i] << " ";
    std::cout << "\nOriginal array - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_array[i] << " ";

    // Вывод результатов CPU
    std::cout << "\n\nCPU processed - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_cpu[i] << " ";
    std::cout << "\nCPU processed - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_cpu[i] << " ";

    // Вывод результатов GPU
    std::cout << "\n\nGPU processed - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_gpu[i] << " ";
    std::cout << "\nGPU processed - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_gpu[i] << " ";

    // Вывод результатов гибридного метода
    std::cout << "\n\nHybrid processed - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_hybrid[i] << " ";
    std::cout << "\nHybrid processed - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_hybrid[i] << " ";

    // ---------------- СРАВНЕНИЕ ВРЕМЕНИ ----------------
    std::cout << "\n\nArray size: " << N << std::endl;
    std::cout << "CPU time (ms): " << cpu_time.count() << std::endl;
    std::cout << "GPU time (ms): " << gpu_time << std::endl;
    std::cout << "Hybrid time (ms): " << hybrid_time << std::endl;

    return 0; // Завершение программы
}

Overwriting assignment4_task3.cu


In [15]:
!nvcc -arch=sm_75 assignment4_task3.cu -o task3
!./task3

Original array - first 10: -119 481 -75 732 -686 846 678 925 -919 -561 
Original array - last 10: -250 -958 -186 848 -452 957 -16 -602 -431 934 

CPU processed - first 10: -238 962 -150 1464 -1372 1692 1356 1850 -1838 -1122 
CPU processed - last 10: -500 -1916 -372 1696 -904 1914 -32 -1204 -862 1868 

GPU processed - first 10: -238 962 -150 1464 -1372 1692 1356 1850 -1838 -1122 
GPU processed - last 10: -500 -1916 -372 1696 -904 1914 -32 -1204 -862 1868 

Hybrid processed - first 10: -238 962 -150 1464 -1372 1692 1356 1850 -1838 -1122 
Hybrid processed - last 10: -500 -1916 -372 1696 -904 1914 -32 -1204 -862 1868 

Array size: 1000000
CPU time (ms): 4.94592
GPU time (ms): 0.131456
Hybrid time (ms): 3.6416


In [16]:
%%writefile assignment4_task3_hybrid.cu

#include <iostream>
#include <vector>
#include <chrono>
#include <random>
#include <thread>           // Библиотека для работы с потоками CPU (std::thread)
#include <cuda_runtime.h>

#define N 1000000           // Размер массива
#define BLOCK_SIZE 256      // Потоков на блок
#define RAND_MIN_VAL -1000
#define RAND_MAX_VAL 1000

// ---------------- GPU Kernel ----------------
// Ядро для параллельных вычислений на видеокарте
__global__ void processKernel(int* d_array, int n) {
    // Вычисляем глобальный индекс текущего потока
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    // Проверка выхода за границы массива
    if (gid < n) {
        d_array[gid] *= 2; // Умножаем элемент на 2
    }
}

// ---------------- CPU processing function ----------------
// Функция для обработки данных на CPU (будет запущена в отдельном потоке)
void processCPU(std::vector<int>& arr, int start, int end) {
    // Обработка заданного диапазона [start, end)
    for (int i = start; i < end; i++) {
        arr[i] *= 2;
    }
}

int main() {
    // ---------------- INIT ARRAY ----------------
    // Подготовка векторов для хранения исходных данных и результатов разных тестов
    std::vector<int> h_array(N), h_cpu(N), h_gpu(N), h_hybrid(N);

    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    // Инициализация исходного массива случайными числами
    for (int i = 0; i < N; i++) {
        h_array[i] = dist(gen);
    }

    // ---------------- CPU PROCESS ----------------
    h_cpu = h_array; // Копируем данные для чистого CPU-теста
    auto cpu_start = std::chrono::high_resolution_clock::now();
    processCPU(h_cpu, 0, N); // Обрабатываем весь массив на CPU последовательно
    auto cpu_end = std::chrono::high_resolution_clock::now();
    // Расчет времени выполнения на CPU
    std::chrono::duration<double, std::milli> cpu_time = cpu_end - cpu_start;

    // ---------------- GPU PROCESS ----------------
    h_gpu = h_array; // Копируем данные для чистого GPU-теста
    int* d_array;
    // Выделяем память на видеокарте и копируем туда весь массив
    cudaMalloc(&d_array, N * sizeof(int));
    cudaMemcpy(d_array, h_gpu.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    // Считаем количество блоков для покрытия N элементов
    int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Создание и фиксация событий для замера времени GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Запуск ядра для обработки всего массива на GPU
    processKernel<<<numBlocks, BLOCK_SIZE>>>(d_array, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop); // Ждем завершения расчетов

    // Копируем результат обратно в h_gpu
    cudaMemcpy(h_gpu.data(), d_array, N * sizeof(int), cudaMemcpyDeviceToHost);

    float gpu_time = 0.0f;
    cudaEventElapsedTime(&gpu_time, start, stop); // Получаем время работы GPU в мс
    cudaFree(d_array); // Освобождаем память на видеокарте

    // ---------------- PARALLEL HYBRID PROCESS ----------------

    h_hybrid = h_array; // Копируем данные для гибридного теста
    int split = N / 2;  // Точка разделения нагрузки пополам

    auto hybrid_start = std::chrono::high_resolution_clock::now();

    // ЗАПУСК ПОТОКА CPU: обрабатывает первую половину массива [0, split)
    // std::ref используется для передачи вектора по ссылке
    std::thread cpu_thread(processCPU, std::ref(h_hybrid), 0, split);

    // ЗАПУСК GPU В ОСНОВНОМ ПОТОКЕ: обрабатывает вторую половину [split, N)
    int* d_hybrid;
    // Выделяем память на GPU только под вторую половину
    cudaMalloc(&d_hybrid, (N - split) * sizeof(int));
    // Копируем вторую половину данных на GPU
    cudaMemcpy(d_hybrid, &h_hybrid[split], (N - split) * sizeof(int), cudaMemcpyHostToDevice);

    // Расчет количества блоков для половины массива
    int blocks_hybrid = ((N - split) + BLOCK_SIZE - 1) / BLOCK_SIZE;
    // Вызываем ядро для обработки половины данных на GPU
    processKernel<<<blocks_hybrid, BLOCK_SIZE>>>(d_hybrid, N - split);
    cudaDeviceSynchronize(); // Ожидаем завершения вычислений на видеокарте

    // Копируем результат GPU-части обратно в исходный вектор на хосте
    cudaMemcpy(&h_hybrid[split], d_hybrid, (N - split) * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_hybrid); // Освобождаем память GPU

    // СИНХРОНИЗАЦИЯ: Ждем, пока поток CPU завершит свою работу над первой половиной
    cpu_thread.join();

    auto hybrid_end = std::chrono::high_resolution_clock::now();
    // Расчет общего времени гибридной обработки (параллельное исполнение)
    float hybrid_time = std::chrono::duration<double, std::milli>(hybrid_end - hybrid_start).count();

    // ---------------- CHECK RESULTS (Вывод результатов) ----------------
    int print_count = 10;
    std::cout << "Original array - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_array[i] << " ";
    std::cout << "\nOriginal array - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_array[i] << " ";

    std::cout << "\n\nCPU processed - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_cpu[i] << " ";
    std::cout << "\nCPU processed - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_cpu[i] << " ";

    std::cout << "\n\nGPU processed - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_gpu[i] << " ";
    std::cout << "\nGPU processed - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_gpu[i] << " ";

    std::cout << "\n\nHybrid processed - first " << print_count << ": ";
    for (int i = 0; i < print_count; i++) std::cout << h_hybrid[i] << " ";
    std::cout << "\nHybrid processed - last " << print_count << ": ";
    for (int i = N - print_count; i < N; i++) std::cout << h_hybrid[i] << " ";

    // ---------------- TIME RESULTS (Вывод временных показателей) ----------------
    std::cout << "\n\nArray size: " << N << std::endl;
    std::cout << "CPU time (ms): " << cpu_time.count() << std::endl;
    std::cout << "GPU time (ms): " << gpu_time << std::endl;
    std::cout << "Hybrid time (ms): " << hybrid_time << std::endl;

    return 0;
}

Writing assignment4_task3_hybrid.cu


In [17]:
!nvcc -arch=sm_75 assignment4_task3_hybrid.cu -o task3
!./task3

Original array - first 10: 649 -850 214 899 776 -688 -359 -922 -245 473 
Original array - last 10: -998 781 484 -341 -876 111 640 794 856 621 

CPU processed - first 10: 1298 -1700 428 1798 1552 -1376 -718 -1844 -490 946 
CPU processed - last 10: -1996 1562 968 -682 -1752 222 1280 1588 1712 1242 

GPU processed - first 10: 1298 -1700 428 1798 1552 -1376 -718 -1844 -490 946 
GPU processed - last 10: -1996 1562 968 -682 -1752 222 1280 1588 1712 1242 

Hybrid processed - first 10: 1298 -1700 428 1798 1552 -1376 -718 -1844 -490 946 
Hybrid processed - last 10: -1996 1562 968 -682 -1752 222 1280 1588 1712 1242 

Array size: 1000000
CPU time (ms): 3.36086
GPU time (ms): 0.12288
Hybrid time (ms): 2.60477
